In [1]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf

# CMP normalisation
from bioinfokit.analys import norm
nm = norm()

# Pre-processing
from sklearn.preprocessing import MinMaxScaler

In [2]:
model = keras.models.load_model('./ErythrOrigin.keras')

2024-08-22 14:43:11.377745: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-08-22 14:43:11.377768: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-22 14:43:11.377776: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-22 14:43:11.377794: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-22 14:43:11.377808: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Undetermined Erythroid cells for prediction
pred = pd.read_csv("Ery_Undetermined_DBEC_MolsPerCell.csv",
                 skiprows = 7,
                 low_memory=False)
pred_Ery = pred[pred['ALAS2'] > 0]
pred_Ery.set_index(['Cell_Index'], inplace=True)
pred = pred_Ery
nm = norm()
nm.cpm(df=pred)
pred = nm.cpm_norm
pred = pred.fillna(0)
pred = pred.replace({0:1})
pred = pred.map(np.log)
# min-max scaling / fit-transform (ft)
pred_ft = MinMaxScaler().fit_transform(pred)
# ndarray to pd.df
pred_ft = pd.DataFrame(pred_ft, columns = pred.columns, index=pred.index)
pred_ft

,ADA,ADGRE1,ADGRG3,AIM2,ALAS2,ANXA5,AOC3,APOBEC3G,APOE,AQP9,...,VNN2,VPREB3,VPS28,VSIG4,XBP1,YBX3,ZAP70,ZBED2,ZBTB16,ZNF683
Cell_Index,,,,,,,,,,,,,,,,,,,,,
270383,0.0,0.0,0.0,0.0,0.165443,1.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.857126,0.0,0.883400,0.356037,0.0,0.0,1.0,0.0
740935,0.0,0.0,0.0,0.0,0.427663,0.966278,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.883400,0.797388,0.0,0.0,0.0,0.0
430513,0.0,0.0,0.0,0.0,0.427663,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.857126,0.0,0.921139,0.535086,0.0,0.0,0.0,0.0
270631,0.0,0.0,0.0,0.0,0.524440,0.966278,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.921139,0.731306,0.0,0.0,0.0,0.0
449078,0.0,0.0,0.0,0.0,0.165443,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.857126,0.0,0.818886,0.469004,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855820,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.581971,0.0,0.0,0.0,0.0
791803,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.618339,0.0,0.0,0.0,0.0
381646,0.0,0.0,0.0,0.0,0.384146,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.356037,0.0,0.0,0.0,0.0


In [10]:
predictions = model.predict(pred_ft)
df_predictions = pd.DataFrame({'Sample_Adult Bone Marrow': predictions[:, 0],
                               'Sample_Cord Blood': predictions[:, 1],
                               'Sample_Fetal Liver': predictions[:, 2],
                               'Sample_Negative Control': predictions[:, 3]})
df_predictions = np.around(df_predictions)
df_predictions.index=pred_ft.index
df_predictions = pd.merge(pred_Ery, df_predictions, left_index=True, right_index=True)
df_predictions = df_predictions.astype(int)
df_predictions

281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


,ADA,ADGRE1,ADGRG3,AIM2,ALAS2,ANXA5,AOC3,APOBEC3G,APOE,AQP9,...,XBP1,YBX3,ZAP70,ZBED2,ZBTB16,ZNF683,Sample_Adult Bone Marrow,Sample_Cord Blood,Sample_Fetal Liver,Sample_Negative Control
Cell_Index,,,,,,,,,,,,,,,,,,,,,
270383,0,0,0,0,2,3,0,0,0,0,...,2,1,0,0,1,0,0,0,0,1
740935,0,0,0,0,6,2,0,0,0,0,...,2,15,0,0,0,0,0,0,1,0
430513,0,0,0,0,6,0,0,0,0,0,...,3,3,0,0,0,0,0,0,0,1
270631,0,0,0,0,9,2,0,0,0,0,...,3,10,0,0,0,0,0,0,1,0
449078,0,0,0,0,2,0,0,0,0,0,...,1,2,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855820,0,0,0,0,1,0,0,0,0,0,...,0,4,0,0,0,0,0,0,0,1
791803,0,0,0,0,1,0,0,0,0,0,...,0,5,0,0,0,0,0,0,0,1
381646,0,0,0,0,5,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [5]:
# Export UMI with the predicted tissue of Erythroid Cell origin
df_predictions.to_csv('Eryhtroid Cells with the Predicted Origin.csv')

## Stats

In [6]:
# Percentage Adult Bone Marrow Erythroid Cell
df_predictions['Sample_Adult Bone Marrow'].mean()

0.0007787295583490933

In [7]:
# Percentage Fetal Liver Erythroid Cell
df_predictions['Sample_Fetal Liver'].mean()

0.09745244187340082

In [8]:
# Percentage Cord Blood Erythroid Cell
df_predictions['Sample_Cord Blood'].mean()

0.8834130604071643

In [9]:
# Percentage unpredicted
df_predictions['Sample_Negative Control'].mean()

0.014684614528868618